#Fine-Tuning GPT-4o-mini
Copyright 2024 Denis Rothman

**August 15,2025 Update**

OpenAI is continually evolving. The model in this notebook is no longer supported.

Please now use **`Chapter08/Fine_tuning_GPT_4_1_mini_SQuAd.ipynb`** that you can access through the README file or directly in the GitHub directory.This notebook is no longer supported.

[OpenAI fine-tuning documentation](https://beta.openai.com/docs/guides/fine-tuning/)

Check the cost of fine-tuning your dataset on OpenAI before running the notebook.

Run this notebook cell by cell to:

1.Download and prepare the SQuAD dataset
Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset.    
2.Fine-tune a model   
3.Run a fine-tuned model

# Installing the environment


In [ ]:
#You can retrieve your API key from a file(1)
# or enter it manually(2)
#Comment this cell if you want to enter your key manually.
#(1)Retrieve the API Key from a file
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
from google.colab import drive
drive.mount('/content/drive')
f = open("drive/MyDrive/files/api_key.txt", "r")
API_KEY=f.readline()
f.close()

In [ ]:
try:
  import openai
except:
  !pip install openai==1.42.0
  import openai

In [ ]:
#(2) Enter your manually by
# replacing API_KEY by your key.
#The OpenAI Key
import os
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
!pip install jsonlines==4.0.0

In [ ]:
!pip install datasets==2.20.0

Listing the installed packages

In [ ]:
import subprocess

# Run pip list and capture the output
result = subprocess.run(['pip', 'list'], stdout=subprocess.PIPE, text=True)

# Split the output into lines and count them
package_list = result.stdout.split('\n')

# Adjust count for headers or empty lines
package_count = len([line for line in package_list if line.strip() != '']) - 2

print(f"Number of installed packages: {package_count}")

In [ ]:
import subprocess

# Run pip list and capture the output
result = subprocess.run(['pip', 'list'], stdout=subprocess.PIPE, text=True)

# Print the output
print(result.stdout)

counting the number of packages

# 1.Preparing the dataset for fine-tuning

## 1.1.Downloading and displaying the dataset

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load the SQuAD dataset from HuggingFace
dataset = load_dataset("squad", split="train[:500]")

# Filter the dataset to ensure context and answer are present
filtered_dataset = dataset.filter(lambda x: x["context"] != "" and x["answers"]["text"] != [])

# Extract prompt (context + question) and response (answer)
def extract_prompt_response(example):
    return {
        "prompt": example["context"] + " " + example["question"],
        "response": example["answers"]["text"][0]  # Take the first answer
    }

filtered_dataset = filtered_dataset.map(extract_prompt_response)

# Print the number of examples
print("Number of examples: ", len(filtered_dataset))

In [ ]:
# Convert the filtered dataset to a pandas DataFrame
df_view = pd.DataFrame(filtered_dataset)

# Display the DataFrame
df_view.head()

## 1.2A Streaming the output to JSON


In [ ]:
import json
import pandas as pd

## 1.2. Preparing the dataset for fine-tuning

In [ ]:
import jsonlines
import pandas as pd
from datasets import load_dataset

# Convert to DataFrame and clean
df = pd.DataFrame(filtered_dataset)
#columns_to_drop = ['title','question','answers']
#df = df.drop(columns=columns_to_drop)

# Prepare the data items for JSON lines file
items = []
for idx, row in df.iterrows():
    detailed_answer = row['response'] + " Explanation: " + row['context']
    items.append({
        "messages": [
            {"role": "system", "content": "Given a SQuAD question built from Wikipedia with crowdworders, provide the correct answer with a detailed explanation."},
            {"role": "user", "content": row['question']},
            {"role": "assistant", "content": detailed_answer}
        ]
    })

# Write to JSON lines file
with jsonlines.open('/content/QA_prompts_and_completions.json', 'w') as writer:
    writer.write_all(items)

### Visualizing the JSON file

In [ ]:
dfile="/content/QA_prompts_and_completions.json"

In [ ]:
import pandas as pd

# Load the data
df = pd.read_json(dfile, lines=True)
df

# 2.Fine-tuning the model



In [ ]:
from openai import OpenAI
import jsonlines
client = OpenAI()
# Uploading the training file

result_file = client.files.create(
  file=open("QA_prompts_and_completions.json", "rb"),
  purpose="fine-tune"
)

print(result_file)
param_training_file_name = result_file.id
print(param_training_file_name)

# Creating the fine-tuning job

ft_job = client.fine_tuning.jobs.create(
  training_file=param_training_file_name,
  model="gpt-4o-mini-2024-07-18"
)

# Printing the fine-tuning job
print(ft_job)

## Monitoring the fine-tunes

In [ ]:
import pandas as pd
from openai import OpenAI
client = OpenAI()
# Assume client is already set up and authenticated
response = client.fine_tuning.jobs.list(limit=3)# increase to see history

# Initialize lists to store the extracted data
job_ids = []
created_ats = []
statuses = []
models = []
training_files = []
error_messages = []
fine_tuned_models = []  # List to store the fine-tuned model names

# Iterate over the jobs in the response
for job in response.data:
    job_ids.append(job.id)
    created_ats.append(job.created_at)
    statuses.append(job.status)
    models.append(job.model)
    training_files.append(job.training_file)
    error_message = job.error.message if job.error else None
    error_messages.append(error_message)

    # Append the fine-tuned model name
    fine_tuned_model = job.fine_tuned_model if hasattr(job, 'fine_tuned_model') else None
    fine_tuned_models.append(fine_tuned_model)

# Create a DataFrame
df = pd.DataFrame({
    'Job ID': job_ids,
    'Created At': created_ats,
    'Status': statuses,
    'Model': models,
    'Training File': training_files,
    'Error Message': error_messages,
    'Fine-Tuned Model': fine_tuned_models  # Include the fine-tuned model names
})

# Convert timestamps to readable format
df['Created At'] = pd.to_datetime(df['Created At'], unit='s')
df = df.sort_values(by='Created At', ascending=False)

# Display the DataFrame
df

### Make sure to obtain your fine-tune model here

If your OpenAI notifications are activated you should receive an email.

Otherwise run the "Monitoring the fine-tunes" cell above to check the status of your fine-tune job.

In [ ]:
import pandas as pd

generation=False  # False until the last model fine-tuned is found. Make sure it used the dataset you trained it on!
# Attempt to find the first non-empty Fine-Tuned Model
non_empty_models = df[df['Fine-Tuned Model'].notna() & (df['Fine-Tuned Model'] != '')]

if not non_empty_models.empty:
    first_non_empty_model = non_empty_models['Fine-Tuned Model'].iloc[0]
    print("The latest fine-tuned model is:", first_non_empty_model)
    generation=True
else:
    first_non_empty_model='None'
    print("No fine-tuned models found.")

In [ ]:
# Fine-tuned model found(True) or not(False)
generation

*Note:* Only continue to Step 3, to use the fine-tuned model when your fine-tuned model is ready. If your OpenAI notifications is activiated, you will receive an email with the status of your fine-tunning job.

# 3.Using the fine-tuned OpenAI model

Note: The is a fine-tuning. As such, be patient!
Rune the `Monitoring the fine-tunes` cell and the f`irst_non_empty_model` cell from time to time.

If the fine-tunning succeeded and your model is ready, the name of your model will be `first_non_empty_model`

1.Go to the OpenAI Playground to test your model: https://platform.openai.com/playground

2.Check the metrics in the fine-tuning UI:
https://platform.openai.com/finetune/

3.Try the fined-tune model out in the cell below.

In [ ]:
# Define the prompt
prompt="Which prize did Frederick Buechner create?"

*Note:* Only run the following cell if your fine-tune job has succeeded and a fined-tuned model is found in the *Monitoring the fine-tunes"* section of *2.Fine-tuning the model.*

In [ ]:
# Assume first_non_empty_model is defined above this snippet
if generation==True:
    response = client.chat.completions.create(
        model=first_non_empty_model,
        temperature=0.0,  # Adjust as needed for variability
        messages=[
            {"role": "system", "content": "Given a question, reply with a complete explanation for students."},
            {"role": "user", "content": prompt}
        ]
    )
else:
    print("Error: Model is None, cannot proceed with the API request.")

In [ ]:
if generation==True:
  print(response)

In [ ]:
if (generation==True):
  # Access the response from the first choice
  response_text = response.choices[0].message.content
  # Print the response
  print(response_text)

In [ ]:
import textwrap

if generation==True:
  wrapped_text = textwrap.fill(response_text.strip(), 60)
  print(wrapped_text)

[Consult OpenAI fine-tune documentation for more](https://platform.openai.com/docs/guides/fine-tuning)